databricks notebook5: create Location dimention table

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id, lit

In [ ]:
# trip silver path
path = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/trip_transactions"
trip_df = spark.read.format('delta').load(path)

In [ ]:
# extract pickup location data
pickup_df = trip_df.select(
     trip_df.source_location_address1.alias("address"),
    trip_df.source_city.alias("city"),
    trip_df.source_province_state.alias("province_state"),
    trip_df.source_country.alias("country")
).withColumn('location_type',lit('pickup'))

In [ ]:
dropoff_df = trip_df.select(
    trip_df.destination_location_address1.alias("address"),
    trip_df.destination_city.alias("city"),
    trip_df.destination_province_state.alias("province_state"),
    trip_df.destination_country.alias("country")
).withColumn('location_type',lit('dropoff'))

In [ ]:
# union them
location_df = pickup_df.union(dropoff_df).dropDuplicates(['address','city','province_state',
                                                          'country','location_type'])

In [ ]:
# Add surrogate LocationID
dim_location = location_df.withColumn("KeylocationID", monotonically_increasing_id())

In [ ]:
# reorder columns
dim_location = dim_location.select('KeylocationID','address','city','province_state',
                                                          'country','location_type')

In [ ]:
# save to silver layer
dim_location.write.format("delta").mode("overwrite").save("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_location")